# Pandas IO Tools: Reading and Writing DataFrames as Files and Databases

## Miroslav Šedivý (@eumiro)
## PyCon.DE 2018

1. CSV
2. JSON
3. HTML
4. Pickle
5. Msgpack
6. Excel
7. HDF5
8. Parquet/Feather
9. PostgreSQL/SQLite

In [3]:
import sys
import pandas as pd
print(f"Python: {sys.version}")
print(f"Pandas: {pd.__version__}")

Python: 3.7.0 (default, Sep 15 2018, 19:13:07) 
[GCC 8.2.1 20180831]
Pandas: 0.23.4


In [4]:
dfr = pd.DataFrame({'a': [1., 2., 3.], 'b': [4., pd.np.nan, 6.]},
                   index=pd.DatetimeIndex(start='2018-10-24', periods=3, freq='1D', name='dt'))

## 1. CSV

In [1]:
!cat a.csv

what
10
11
12


In [8]:
pd.read_csv('a.csv')

,what
0,10
1,11
2,12


In [25]:
print(dfr.to_csv())

,a,b
2018-10-24,1.0,4.0
2018-10-25,2.0,
2018-10-26,3.0,6.0



## 2. JSON

In [10]:
dfr.to_json()

'{"a":{"1540339200000":1.0,"1540425600000":2.0,"1540512000000":3.0},"b":{"1540339200000":4.0,"1540425600000":null,"1540512000000":6.0}}'

## 3. HTML

In [9]:
url = 'https://www.wetterdienst.de/Deutschlandwetter/Karlsruhe_(Baden)/Aktuell/'
pd.read_html(url)

[                0           1       2             3           4  \
 0             NaN  Temperatur  Wetter  Niederschlag   Luftdruck   
 1   Di, 22:00 Uhr         14°     NaN             -  1026.3 hPa   
 2   Di, 21:30 Uhr         14°     NaN             -  1026.5 hPa   
 3   Di, 21:20 Uhr         14°     NaN          0 mm  1026.5 hPa   
 4   Di, 21:10 Uhr         14°     NaN          0 mm  1026.4 hPa   
 5   Di, 21:00 Uhr         14°     NaN          0 mm    1026 hPa   
 6   Di, 20:50 Uhr         14°     NaN          0 mm    1026 hPa   
 7   Di, 20:40 Uhr         14°     NaN          0 mm  1025.9 hPa   
 8   Di, 20:30 Uhr         14°     NaN          0 mm  1025.9 hPa   
 9   Di, 20:20 Uhr         14°     NaN          0 mm  1025.9 hPa   
 10  Di, 20:10 Uhr         14°     NaN          0 mm  1025.6 hPa   
 11  Di, 20:00 Uhr         14°     NaN          0 mm  1025.4 hPa   
 12  Di, 19:50 Uhr         14°     NaN          0 mm  1025.4 hPa   
 13  Di, 19:40 Uhr         14°     NaN          

In [ ]:
dfr.to_html()

## 4. Pickle

In [35]:
# protocol
# compression

# read/write

## 5. Msgpack

In [ ]:
pd.read_msgpack()

## 6. Excel

http://pbpython.com/improve-pandas-excel-output.html

In [15]:
reader = pd.ExcelFile('pandasio.xlsx')
dfr = pd.read_excel(reader)
w = pd.ExcelWriter('pandasio2.xlsx', engine='xlsxwriter')
dfr.to_excel(w, sheet_name='one')
sh = w.book.worksheets()[0]
sh.write_formula('E10', '=SUM(E2:E9)')
w.save()

## 7. HDF5

In [9]:
dfr.to_hdf('dfr.h5', 'dfr', format='table')

## 8. Parquet/Feather

What are the differences between Feather and Parquet?
https://stackoverflow.com/questions/48083405/what-are-the-differences-between-feather-and-parquet

Wes McKinney (January 2018):
- Parquet format is designed for long-term storage, where Arrow is more intended for short term or ephemeral storage (Arrow may be more suitable for long-term storage after the 1.0.0 release happens, since the binary format will be stable then) _[MŠ: Version 0.11.0 as of 2018-10-08]_
- Parquet is more expensive to write than Feather as it features more layers of encoding and compression. Feather is unmodified raw columnar Arrow memory. We will probably add simple compression to Feather in the future.
- Due to dictionary encoding, RLE encoding, and data page compression, Parquet files will often be much smaller than Feather files
- Parquet is a standard storage format for analytics that's supported by many different systems: Spark, Hive, Impala, various AWS services, in future by BigQuery, etc. So if you are doing analytics, Parquet is a good option as a reference storage format for query by multiple systems


## 9. PostgreSQL/SQLite

In [44]:
import sqlite3
conn = sqlite3.connect('db.sqlite')
dfr.to_sql(con=conn, name='dfr', if_exists='replace')

print(pd.read_sql("SELECT * FROM dfr WHERE dt > CURRENT_TIMESTAMP", con=conn))


                    dt    a    b
0  2018-10-24 00:00:00  1.0  4.0
1  2018-10-25 00:00:00  2.0  NaN
2  2018-10-26 00:00:00  3.0  6.0
